### This notebook has various explorations of the market access tools, but is not desgined as a complete workflow. Use at your own discretion

In [11]:
import sys, os, importlib
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import GOSTnets as gn
import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features
from shapely.geometry import box, Point, Polygon
from scipy.ndimage import generic_filter
from pandana.loaders import osm

sys.path.append("../")

import infrasap.market_access as ma
import infrasap.GOSTRocks.rasterMisc as rMisc
#import infrasap.rai_calculator as rai

In [ ]:
global_friction_surface = "/home/public/Data/GLOBAL/INFRA/FRICTION_2015/2015_friction_surface_v1.geotiff"

inG = rasterio.open(global_friction_surface)


In [3]:
global_admin = "/home/public/Data/GLOBAL/ADMIN/Admin0_Polys.shp"
inD = gpd.read_file(global_admin)
selD = inD.loc[inD['Region'] == "Sub-Saharan Africa"]
selD = selD.to_crs(inG.crs)

,OBJECTID,ISO_A2,WB_ADM0_CO,WB_ADM0_NA,Shape_Leng,Shape_Area,ISO3,UN_m49,Region,incomeG,lendingC,FID_100,geometry
0,1,AF,1,Afghanistan,7.132529e+06,6.418379e+05,AFG,4,South Asia,Low income,IDA,0,"POLYGON ((7903635.422200002 4647361.696500003,..."
1,2,AL,3,Albania,1.743971e+06,2.868177e+04,ALB,8,Europe & Central Asia,Upper middle income,IBRD,100,(POLYGON ((2145902.261799999 4941064.025700003...
2,3,DZ,4,Algeria,8.933841e+06,2.309321e+06,DZA,12,Middle East & North Africa,Upper middle income,IBRD,200,(POLYGON ((-125251.4530000016 4264455.85010000...
3,4,AS,5,American Samoa (U.S.),1.744573e+05,2.110162e+02,ASM,16,East Asia & Pacific,Upper middle income,None,300,"(POLYGON ((-18985804.4762 -1607929.979800001, ..."
4,5,SD,6,Sudan,8.852111e+06,1.844887e+06,SDN,736,Sub-Saharan Africa,Lower middle income,IDA,400,"(POLYGON ((4282246.283599999 2048756.8814, 428..."


In [12]:
# clip out the Africa Raster
out_file = "/home/wb411133/temp/AFR_friction_2015.tif"
rMisc.clipRaster(inG, selD, out_file)

In [13]:
sel_G = rasterio.open(out_file)
inD = sel_G.read()
# inD is stored as minutes to travel one meter. Need to convert to second to traverse the entire 1 km cell
mcp = graph.MCP_Geometric(inD[0,:,:])

In [14]:
in_ports = "/home/public/Data/GLOBAL/INFRA/PORTS/ports_noProv.csv"
out_folder = "/home/wb411133/data/Global/INFRA/PORTS"
ports_file = os.path.join(out_folder, "major_ports.shp")
if not os.path.exists(ports_file):
    inP = pd.read_csv(in_ports)
    inP_geom = [Point(x) for x in zip(inP['Lng'], inP['Lat'])]
    inP = gpd.GeoDataFrame(inP, geometry = inP_geom, crs={'init':'epsg:4326'})
    inP.to_file(ports_file)
else:
    inP = gpd.read_file(ports_file)

In [15]:
importlib.reload(ma)
ma.generate_market_sheds?

In [17]:
importlib.reload(ma)
inG = rasterio.open("/home/wb411133/temp/AFR_friction_2015.tif")
out_file = "/home/wb411133/temp/port_sheds_AFR.tif"
ma.generate_market_sheds(inG, inP, out_file)

MemoryError: 

In [ ]:
ma.generate_feature_vectors?

In [ ]:
inD = inG.read()
# inD is stored as minutes to travel one meter. Need to convert to second to traverse the entire 1 km cell
mcp = graph.MCP_Geometric(inD[0,:,:] * 1000)

In [ ]:
drive_time_thresholds = [1, 5, 10, 15] # days
drive_time_thresholds = [x * 24 * 60 for x in drive_time_thresholds] #convert days to minutes
drive_vectors = ma.generate_feature_vectors(inR, mcp, inD, drive_time_thresholds)
drive_vectors.to_file(os.path.join(tutorial_folder, "drive_vectors.shp"))

# Calculate market sheds from travel datasets
https://scikit-image.org/docs/dev/api/skimage.graph.html#skimage.graph.MCP_Geometric


In [ ]:
import sys, os, importlib
import rasterio

import numpy as np
import pandas as pd
import geopandas as gpd
import osmnx as ox
import GOSTnets as gn
import skimage.graph as graph

from rasterio.mask import mask
from rasterio import features
from shapely.geometry import box, Point, Polygon
from scipy.ndimage import generic_filter
from pandana.loaders import osm

In [ ]:
tutorial_folder ="../tutorial_data"
img = os.path.join(tutorial_folder, "global_friction_surface.tif")
dests = os.path.join(tutorial_folder, "destinations.shp")

img = rasterio.open(img)
inD = gpd.read_file(dests)
data = img.read()[0,:,:]

In [ ]:
mcp = graph.MCP_Geometric(data)
dests_geom = [img.index(x.x, x.y) for x in inD['geometry']]
costs, traceback = mcp.find_costs(dests_geom)

In [ ]:
meta = img.meta.copy()
meta.update(dtype=costs.dtype)

with rasterio.open(os.path.join(tutorial_folder, "market_shed_custom.tif"), 'w', **meta) as outR:
    outR.write_band(1, costs)

# Debugging below

In [ ]:
import copy
# testing market sheds
dests_geom = [img.index(x.x, x.y) for x in inD['geometry']]
all_c = []
n = inD.shape[0]
idx = 0

In [ ]:
for dest in dests_geom:
    idx += 1
    if dest[0] > 0 and dest[0] < img.shape[0] and dest[1] > 0 and dest[1] < img.shape[1]:
        c1, trace = mcp.find_costs([dest])
        all_c.append(copy.deepcopy(c1))
    else:
        print(f"{idx} of {n} cannot be processed")


In [ ]:
# Iterate through results to generate final marketshed
res = np.zeros(all_c[0].shape)
for idx in range(0, len(all_c)):
    cur_res = all_c[idx]
    if idx == 0:
        min_res = cur_res
    else:
        combo = np.dstack([min_res, cur_res])
        min_res = np.amin(combo, 2)
        cur_val = (min_res == cur_res).astype(np.byte)
        m_idx = np.where(cur_val == 1)
        res[m_idx] = idx


In [ ]:
meta = img.meta.copy()
meta.update(dtype=res.dtype)
with rasterio.open(os.path.join(tutorial_folder, "market_shed_custom.tif"), 'w', **meta) as outR:
    outR.write_band(1, res)

In [ ]:
output.shape

In [ ]:
class MarketShedMCP(graph.MCP_Flexible):
    def _reset(self):
        """reset the id map
        """
        graph.MCP_Flexible._reset(self)
        self._conn = {}
        self._bestconn_v = {}
        self._bestconn = {}
    
    def create_connection(self, id1, id2, pos1, pos2, cost1, cost2):
        # Process data
        hash = min(id1, id2), max(id1, id2)
        val = min(pos1, pos2), max(pos1, pos2)
        cost = min(cost1, cost2)
        # Add to total list 
        self._conn.setdefault(hash, []).append(val)
        # Keep track of connection with lowest cost
        curcost = self._bestconn_v.get(hash, (np.inf,))[0]        
        if cost1 < cost2:
            self._bestconn[val] = (id1,)
        else:
            self._bestconn[val] = (id2,)        
        if cost < curcost:            
            self._bestconn_v[hash] = (cost,) + val
    
mcp_m = MarketShedMCP(data)
costs, traceback = mcp_m.find_costs(dests_geom)

In [ ]:
data.shape[0] * data.shape[1]

In [ ]:
len(mcp_m._bestconn.keys())

In [ ]:
graph.MCP_Flexible.update_node?

In [ ]:
costs, traceback = mcp.find_costs(dests_geom, find_all_ends=True)

In [ ]:
meta = img.meta.copy()
meta.update(dtype=res.dtype, count=5)
with rasterio.open(os.path.join(tutorial_folder, "market_shed.tif"), 'w', **meta) as outR:
    for idx in range(0,5):
        outR.write_band(idx + 1, res[:,:,idx])

In [ ]:
def get_min_axis(x):
    return(np.where(x == x.min()))

res_min = np.apply_along_axis(get_min_axis, 2, res)

In [ ]:
meta = img.meta.copy()
res_min = res_min.astype(meta['dtype'])

In [ ]:
res_min.shape

In [ ]:

meta.update(dtype=res_min.dtype)
with rasterio.open(os.path.join(tutorial_folder, "market_shed_2.tif"), 'w', **meta) as outR:
    outR.write_band(1, res_min[:,:,0,0])

'''
meta.update(dtype=costs.dtype)
with rasterio.open(os.path.join(tutorial_folder, "travel_costs_fa.tif"), 'w', **meta) as outR:
    outR.write_band(1, costs)
    
meta.update(dtype=traceback.dtype)
with rasterio.open(os.path.join(tutorial_folder, "traceback_fa.tif"), 'w', **meta) as outR:
    outR.write_band(1, traceback)
'''

In [ ]:
sys.path.append("../")

import infrasap.market_access as ma

In [ ]:
importlib.reload(ma)
out_file = os.path.join(tutorial_folder, "market_shed_3.tif")
ma.generate_market_sheds(img, mcp, inD, out_file)

# Calculate area of cells in raster dataset

In [ ]:
global_friction_surface = "/home/public/Data/GLOBAL/INFRA/FRICTION_2015/2015_friction_surface_v1.geotiff"

inG = rasterio.open(global_friction_surface)
inD = inG.read()

In [ ]:
inD.shape

In [ ]:
inG.bounds

In [ ]:
def generate_shape(x, llx, lly, res):
    lly = lly + (x * res)
    ll = (llx, lly)
    ul = (llx, lly + res)
    ur = (llx + res, lly + res)
    lr = (llx + res, lly)
    shape = Polygon([ll, ul, ur, lr, ll])
    return(shape)
    
b = inG.bounds
res = inG.meta['transform'][0]
all_shapes = [generate_shape(x, b[0], b[1], res) for x in range(0, inD.shape[1])]

In [ ]:
res = pd.DataFrame(columns=["idx"])
res['idx'] = list(range(0, inG.shape[0]))
res = gpd.GeoDataFrame(res, geometry=all_shapes, crs = inG.crs)
res['area'] = res['geometry'].apply(lambda x: x.area)

In [ ]:
res.to_file("/home/wb411133/temp/gfs_column1.shp")

In [ ]:
# Convert to UTM to calculate area to metres
'''sys.path.append("../../GOST")

import GOSTRocks.misc as misc
res_sel = res.iloc[1:-1,]
res_utm = misc.project_UTM(res_sel)'''

res_utm = res.to_crs({'init':'epsg:32601'})

In [ ]:
res_utm['area_utm'] = res_utm['geometry'].apply(lambda x: x.area)
res_utm.to_file("/home/wb411133/temp/gfs_column1.shp")

In [ ]:
res_utm.head()

In [ ]:
import numpy as np
import skimage.graph as graph
import copy

In [ ]:
img = np.array([[1,1,2,2,2],[2,1,1,3,3],[3,2,1,2,3],[2,2,2,1,1]])
mcp = graph.MCP_Geometric(img)

In [ ]:
destinations = [[0,0],[3,3]]
costs, traceback = mcp.find_costs(destinations)
print(costs)
print(traceback)

In [ ]:
output

In [ ]:
all_c = []
for dest in destinations:
    costs, traceback = mcp.find_costs([dest])
    all_c.append(copy.deepcopy(costs))

In [ ]:
res = np.dstack(all_c)
res_min = np.amin(res, axis=2)
output = np.zeros([res_min.shape[0], res_min.shape[1]])
for idx in range(0, res.shape[2]):
    cur_data = res[:,:,idx]
    cur_val = (cur_data == res_min).astype(np.byte) * idx
    output = output + cur_val
output = output.astype(np.byte)


In [ ]:
output

In [ ]:
res[:,:,1]

In [ ]:
class MarketShedMCP(graph.MCP_Geometric):
    def _reset(self):
        """reset the id map
        """
        graph.MCP_Flexible._reset(self)
        self._conn = {}
        self._bestconn_v = {}
        self._bestconn = {}
    
    def create_connection(self, id1, id2, pos1, pos2, cost1, cost2):
        # Process data
        hash = min(id1, id2), max(id1, id2)
        print(hash)
        val = min(pos1, pos2), max(pos1, pos2)
        cost = min(cost1, cost2)
        # Add to total list 
        self._conn.setdefault(hash, []).append(val)
        # Keep track of connection with lowest cost
        curcost = self._bestconn_v.get(hash, (np.inf,))[0]        
        if cost1 < cost2:
            self._bestconn[val] = (id1,)
        else:
            self._bestconn[val] = (id2,)        
        if cost < curcost:            
            self._bestconn_v[hash] = (cost,) + val
    
mcp_m = MarketShedMCP(img)
costs, traceback = mcp_m.find_costs(destinations)

In [ ]:
costs

In [ ]:
import numpy as np
from skimage import graph

image = np.array(
    [[1, 1, 2, 2, 2, 2],
     [2, 1, 1, 3, 3, 3],
     [3, 2, 1, 2, 2, 2],
     [3, 2, 2, 1, 1, 1],
     [4, 3, 2, 1, 1, 4],
     [4, 3, 2, 1, 1, 4]]
)
destinations = [[0, 0], [3, 3]]
mcp = graph.MCP_Geometric(image)
costs, traceback = mcp.find_costs(destinations)

In [ ]:
offsets = _mcp.make_offsets(2, True)
offsets.append(np.array([0, 0]))
offsets_arr = np.array(offsets)
indices = np.indices(traceback.shape)
offset_to_neighbor = offsets_arr[traceback]
neighbor_index = indices - offset_to_neighbor.transpose((2, 0, 1))
ids = np.arange(traceback.size).reshape(costs.shape)
neighbor_ids = np.ravel_multi_index(
    tuple(neighbor_index), traceback.shape
)
g = sparse.coo_matrix((
        np.ones(traceback.size),
        (ids.flat, neighbor_ids.flat)),
    )
g.toarray().shape




In [ ]:
sparse.coo_matrix?

In [ ]:
n, components = sparse.csgraph.connected_components(g, directed=False)
basins = components.reshape(costs.shape)

In [ ]:
n, components = sparse.csgraph.connected_components(ids, directed=False)
basins = components.reshape(costs.shape)

In [ ]:
neighbor_ids